# Mapping part 2

In this notebook we will extend our mapping abilities by plotting some cool data! We are going to do quite a lot of work with gridded data in this notebook, so to start us off we will try plotting a dem.

By the end of this notebook you should know how to contour data on maps, and add vectors. We will demonstrate this by making an isopach map and thinking about how wind-direction affects ejecta distribution.

In [1]:
%matplotlib widget

---
## Topography

Load DEM for whole of NZ and clip to region

In [2]:
import rasterio
import matplotlib.pyplot as plt

dem = rasterio.open("data/nztm.tif")  # Need to copy this from /Volumes/GeoPhysics_09/users-data/chambeca
fig, ax = plt.subplots()
ax.imshow(dem.read()[0], cmap="Greys", vmin=-100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We need to define the region we want to clip to and that region needs to be defined in a way that `rasterio` understands - e.g. it needs to be in a real co-ordinate system.  The way to do this is to define a `Shapely` polygon. [Shapely](https://shapely.readthedocs.io/en/stable/index.html) is a really powerful library for working with shapes - if you ever need to work out whether one thing is in another thing, or shear an arbitrary object (maybe a clast?) and work out where the points of that shape would end up, Shapely will help.

As an aside, I will define a rectanglular polygon, but we could define any shape and we could mask a grid to that shape - you could also use a shape-file and clip to that.

In [3]:
from shapely.geometry import Polygon
from shapely.ops import transform
import pyproj

min_longitude, max_longitude, min_latitude, max_latitude = 173.8, 175.7, -42.0, -40.7

geom = Polygon([
    (min_longitude, min_latitude), 
    (max_longitude, min_latitude),
    (max_longitude, max_latitude), 
    (min_longitude, max_latitude), 
    (min_longitude, min_latitude)])

transformer = pyproj.Transformer.from_proj(
    proj_from=pyproj.Proj("epsg:4326"), 
    proj_to=dem.crs.to_proj4(),
    always_xy=True)  # Set to give in longitude, latitude as above

nztm_geom = transform(transformer.transform, geom)
print(nztm_geom)

POLYGON ((1666254.326417144 5349914.263328397, 1823616.372882802 5346696.852426703, 1828113.381794665 5491038.614971211, 1667585.694441289 5494237.219997164, 1666254.326417144 5349914.263328397))


/home/chambeca/miniconda3/envs/eqcorrscan/lib/python3.8/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


In [4]:
from rasterio.mask import mask

cliped_dem, clip_transform = mask(dem, [nztm_geom], crop=True)
meta_data = dem.meta

meta_data.update({"driver": "GTiff",
                  "height": cliped_dem.shape[1],
                  "width": cliped_dem.shape[2],
                  "transform": clip_transform})

with rasterio.open("data/cliped_dem.tif", "w", **meta_data) as dest:
    dest.write(cliped_dem)

In [5]:
cliped_dem = rasterio.open("data/cliped_dem.tif")
print(cliped_dem)

fig, ax = plt.subplots()
ax.imshow(cliped_dem.read()[0], cmap="Greys", vmin=-20)

<open DatasetReader name='data/cliped_dem.tif' mode='r'>


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
import cartopy.crs as ccrs
import xarray as xr
import numpy as np
from rasterio.warp import transform
import cartopy.feature as cfeature

proj = ccrs.AlbersEqualArea(
    central_latitude=-41.7, 
    central_longitude=174.7, 
    standard_parallels=[-40, -42])

fig = plt.figure()
ax = fig.add_subplot(projection=proj)
ax.set_extent((173.8, 175.7, -42.0, -40.7), crs=ccrs.PlateCarree())


dem = xr.open_rasterio("data/cliped_dem.tif")
ny, nx = len(dem['y']), len(dem['x'])
x, y = np.meshgrid(dem['x'], dem['y'])
lon, lat = transform(dem.crs, {'init': 'EPSG:4326'},
    x.flatten(), y.flatten())
lon = np.asarray(lon).reshape((ny, nx))
lat = np.asarray(lat).reshape((ny, nx))
dem.coords['lon'] = (('y', 'x'), lon)
dem.coords['lat'] = (('y', 'x'), lat)

# greyscale = dem.clip(min=-10).mean(dim='band')
greyscale = dem.clip(min=0)
greyscale.plot(ax=ax, x='lon', y='lat', transform=ccrs.PlateCarree(),
               cmap='Greys', add_colorbar=False)
coast = cfeature.GSHHSFeature(
    scale="h", levels=[1], edgecolor="black")
ax.add_feature(coast)

# ocean = cfeature.NaturalEarthFeature(cfeature.OCEAN.category,
#                                      cfeature.OCEAN.name, "10m",
#                                      edgecolor='face',
#                                      facecolor='none')
# ax.add_feature(ocean, facecolor="white")

ax.gridlines(
    draw_labels=True, 
    xlocs=[174, 174.5, 175, 175.5], 
    ylocs=[-42, -41.5, -41])
ax.set_title("")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '')

---
## Beyond Cartopy

Cartopy and friends are pretty cool, but not this isn't by any means the only way to plot geospatial data. Below are a list of other projects you might want to check out:

- [GMT](https://www.generic-mapping-tools.org/) remains the standard mapping interface for most geophysics and geology publications, and makes really pretty plots. [pyGMT](https://www.pygmt.org/latest/) (GMT for Python) is coming, but developing the interfaces takes quite a while - for most of us writing code isn't our main job, so developing has to fit around the other things that we are paid to do (teach, do research, write proposals, moan about management etc.)
- [bokeh](https://bokeh.org/) is pretty cool for creating interactive plots, including [maps](https://docs.bokeh.org/en/latest/docs/user_guide/geo.html) - CJC uses this quite a bit for interogating data.
- [follium](https://github.com/python-visualization/folium) looks nice for making interactive mapping plots - possibly better than bokeh because it is more focused on mapping.

That is all for now - hopefully that has given you some idea of how you can make some fairly nice plots without too much effort.  A key idea with these notebooks is for you to **borrow some code** from them so that you do not have to re-write everything yourself.